Test different functions to get the core microbiota:

In [4]:
import os
import pandas as pd
from qiime2 import Visualization
import matplotlib.pyplot as plt
import numpy as np

import qiime2 as q2

%matplotlib inline
data_dir = 'CE'

##### Download metadata

In [14]:
! wget -nv -O $data_dir/food-metadata.tsv 'https://polybox.ethz.ch/index.php/s/nEd4l5CWGWGEtae/download'

2022-12-12 08:54:46 URL:https://polybox.ethz.ch/index.php/s/nEd4l5CWGWGEtae/download [42810/42810] -> "CE/food-metadata.tsv" [1]


Identify "core" features, which are features observed in a user-defined
  fraction of the samples. Since the core features are a function of the
  fraction of samples that the feature must be observed in to be considered
  core, this is computed over a range of fractions defined by the
  `min_fraction`, `max_fraction`, and `steps` parameters.

#### Workflow
1) Try different parameters to find core features
2) Find core features of all cheeses in our feature table
3) Find core features of Swiss cheeses (in categories rindtype = natural, washed or style = alpine
4) Find core features of similar neighboring country cheeses.
5) Compare results of Swiss to neighboring country cheeses.

### 1)

I tried different values for the parameters:

#### 1. Try

Used the function with the default values:

In [9]:
! qiime feature-table core-features \
--i-table $data_dir/dada2_table_align_filtered.qza \
--p-min-fraction 0.5 \
--o-visualization $data_dir/core_microbiota.qzv

Usage: qiime feature-table core-features [OPTIONS]

  Identify "core" features, which are features observed in a user-defined
  fraction of the samples. Since the core features are a function of the
  fraction of samples that the feature must be observed in to be considered
  core, this is computed over a range of fractions defined by the
  `min_fraction`, `max_fraction`, and `steps` parameters.

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency]
                       The feature table to use in core features
                       calculations.                                [required]
Parameters:
  --p-min-fraction PROPORTION Range(0.0, 1.0, inclusive_start=False)
                       The minimum fraction of samples that a feature must be
                       observed in for that feature to be considered a core
                       feature.                                 [default: 0.5]
  --p-max-fraction PROPORTION Range(0.0, 1.0, inclusive_end=True)
                       

In [4]:
Visualization.load(f'{data_dir}/core_microbiota.qzv')

<visualization: Visualization uuid: 0c0156f8-5401-48d1-80c0-988547d889ce>

#### 2. Try

Used the function with higher min-fraction:

In [14]:
! qiime feature-table core-features \
--i-table $data_dir/dada2_table_align_filtered.qza \
--p-min-fraction 0.8 \
--o-visualization $data_dir/core_microbiota_2.qzv

Saved Visualization to: CE/core_microbiota_2.qzv


In [15]:
Visualization.load(f'{data_dir}/core_microbiota_2.qzv')

<visualization: Visualization uuid: 61f25892-088c-472b-9bc4-bcad74e4e50a>

#### 3. Try

Using different step value:

In [17]:
! qiime feature-table core-features \
--i-table $data_dir/dada2_table_align_filtered.qza \
--p-min-fraction 0.8 \
--p-steps 5 \
--o-visualization $data_dir/core_microbiota_3.qzv

Saved Visualization to: CE/core_microbiota_3.qzv


In [18]:
Visualization.load(f'{data_dir}/core_microbiota_3.qzv')

<visualization: Visualization uuid: 6fe15849-2827-4fbf-969f-d4dae15e4cb2>

#### 4. Try

Use different min-fraction:

In [22]:
! qiime feature-table core-features \
--i-table $data_dir/dada2_table_align_filtered.qza \
--p-min-fraction 0.7 \
--p-steps 10 \
--o-visualization $data_dir/core_microbiota_4.qzv

Saved Visualization to: CE/core_microbiota_4.qzv


In [2]:
Visualization.load(f'{data_dir}/core_microbiota_4.qzv')

<visualization: Visualization uuid: 78f37bc9-1287-4d66-bc69-3969c384e827>

### 2)

#### Download tsv file

TSV file with feature list could be downloaded from the output above. I downloaded the file and put it on polybox. I set the treshold of fraction of samples (The fraction of the total number of samples that a feature must be observed in for that feature to be considered "core".) to 0.7.
Here we import this data from the polybox:

In [11]:
! wget -nv -O $data_dir/core_microbiota_list_0.7.tsv 'https://polybox.ethz.ch/index.php/s/WRm86jdxvkxPOVa/download'

2022-12-12 08:53:38 URL:https://polybox.ethz.ch/index.php/s/WRm86jdxvkxPOVa/download [490/490] -> "CE/core_microbiota_list_0.7.tsv" [1]


This are the core features of all cheeses:

In [12]:
df_core_all = pd.read_csv(f'{data_dir}/core_microbiota_list_0.7.tsv', sep ='\t')
df_core_all

,Feature ID,2%,9%,25%,50%,75%,91%,98%
0,f50c8ae2717bb99c926c4ab1f2a6135c,4.0,12.0,184.5,1897.5,8307.25,51957.23,88576.02
1,805c1b3ec3035abbb7b9f1f7f6157e12,0.0,13.0,98.5,741.0,6608.00,21851.37,102426.32
2,5899b66b70d688d5cd95df5fc7a26e3a,0.0,0.0,8.0,87.0,1019.25,6905.73,28623.58
3,369232e1ac9f9983056d09b9fe866df5,0.0,0.0,8.0,44.0,400.75,2877.72,12945.78
4,398e906d9ad1914eb268fda5c7453e09,0.0,3.0,6.0,32.0,1070.25,11938.72,47885.18


### 3) 

Do cheeses from
Switzerland share this core microbiome with similar cheeses (e.g., same style/rind type) from neighboring
countries?

Find core features of CH cheeses with natural rindtype:

Result: 33 core features

In [22]:
! qiime feature-table filter-samples \
--i-table $data_dir/dada2_table_align_filtered.qza \
--m-metadata-file $data_dir/food-metadata.tsv \
--p-where "[country]='Switzerland' AND [rindtype]='natural'"\
--o-filtered-table $data_dir/feature_table_CH_natural.qza

Saved FeatureTable[Frequency] to: CE/feature_table_CH_natural.qza


In [29]:
! qiime feature-table core-features \
--i-table $data_dir/feature_table_CH_natural.qza \
--p-min-fraction 0.7 \
--p-steps 10 \
--o-visualization $data_dir/core_microbiota_CH_natural.qzv

Saved Visualization to: CE/core_microbiota_CH_natural.qzv


In [30]:
Visualization.load(f'{data_dir}/core_microbiota_CH_natural.qzv')

<visualization: Visualization uuid: 943f3266-4c8c-4004-bba8-65dbcb0ebf16>

Find core features of CH cheeses with washed rindtype:

In [31]:
! qiime feature-table filter-samples \
--i-table $data_dir/dada2_table_align_filtered.qza \
--m-metadata-file $data_dir/food-metadata.tsv \
--p-where "[country]='Switzerland' AND [rindtype]='washed'"\
--o-filtered-table $data_dir/feature_table_CH_washed.qza

Saved FeatureTable[Frequency] to: CE/feature_table_CH_washed.qza


In [32]:
! qiime feature-table core-features \
--i-table $data_dir/feature_table_CH_washed.qza \
--p-min-fraction 0.7 \
--p-steps 10 \
--o-visualization $data_dir/core_microbiota_CH_washed.qzv

Saved Visualization to: CE/core_microbiota_CH_washed.qzv


In [33]:
Visualization.load(f'{data_dir}/core_microbiota_CH_washed.qzv')

<visualization: Visualization uuid: 44aaefdd-61c0-4a7d-bded-3879ace69aba>

Find core features of CH cheeses with alpine style:

In [35]:
! qiime feature-table filter-samples \
--i-table $data_dir/dada2_table_align_filtered.qza \
--m-metadata-file $data_dir/food-metadata.tsv \
--p-where "[country]='Switzerland' AND [style]='alpine'"\
--o-filtered-table $data_dir/feature_table_CH_alpine.qza

Saved FeatureTable[Frequency] to: CE/feature_table_CH_alpine.qza


In [36]:
! qiime feature-table core-features \
--i-table $data_dir/feature_table_CH_alpine.qza \
--p-min-fraction 0.7 \
--p-steps 10 \
--o-visualization $data_dir/core_microbiota_CH_alpine.qzv

Saved Visualization to: CE/core_microbiota_CH_alpine.qzv


In [37]:
Visualization.load(f'{data_dir}/core_microbiota_CH_alpine.qzv')

<visualization: Visualization uuid: c060248a-b70e-4c1a-9d20-f0defc56c631>

In [23]:
! qiime feature-table summarize \
--i-table $data_dir/feature_table_CH_natural.qza \
--o-visualization $data_dir/feature_table_CH_natural.qzv

Saved Visualization to: CE/feature_table_CH_natural.qzv


In [24]:
Visualization.load(f'{data_dir}/feature_table_CH_natural.qzv')

<visualization: Visualization uuid: 1723da06-c9e2-4655-b698-beb5f16d91b6>

In [17]:
! qiime feature-table summarize --help

Usage: qiime feature-table summarize [OPTIONS]

  Generate visual and tabular summaries of a feature table.

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency | RelativeFrequency |
    PresenceAbsence]   The feature table to be summarized.          [required]
Parameters:
  --m-sample-metadata-file METADATA...
    (multiple          The sample metadata.
     arguments will    
     be merged)                                                     [optional]
Outputs:
  --o-visualization VISUALIZATION
                                                                    [required]
Miscellaneous:
  --output-dir PATH    Output unspecified results to a directory
  --verbose / --quiet  Display verbose output to stdout and/or stderr during
                       execution of this action. Or silence output if
                       execution is successful (silence is golden).
  --example-data PATH  Write example data and exit.
  --citations          Show citations and exit.
  --help               

In [2]:
! qiime feature-table filter-samples --help

Usage: qiime feature-table filter-samples [OPTIONS]

  Filter samples from table based on frequency and/or metadata. Any features
  with a frequency of zero after sample filtering will also be removed. See
  the filtering tutorial on https://docs.qiime2.org for additional details.

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency¹ | RelativeFrequency² |
    PresenceAbsence³ | Composition⁴]
                       The feature table from which samples should be
                       filtered.                                    [required]
Parameters:
  --p-min-frequency INTEGER
                       The minimum total frequency that a sample must have to
                       be retained.                               [default: 0]
  --p-max-frequency INTEGER
                       The maximum total frequency that a sample can have to
                       be retained. If no value is provided this will default
                       to infinity (i.e., no maximum frequency filter will 